<a href="https://colab.research.google.com/github/yeonghun00/Twitter-Sentiment-Analysis/blob/main/Market%20Sentiment%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nest-asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nest_asyncio 
nest_asyncio.apply()

In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install twint
!pip3 install --user --upgrade "git+https://github.com/twintproject/twint.git@origin/master#egg=twint"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/twintproject/twint.git (to revision origin/master) to /tmp/pip-install-m60lkj6h/twint_e8077e07edc94660994f75dcb7e74256
  Running command git clone --filter=blob:none --quiet https://github.com/twintproject/twint.git /tmp/pip-install-m60lkj6h/twint_e8077e07edc94660994f75dcb7e74256
  Running command git checkout -q origin/master
  Resolved https://github.com/twintproject/twint.git to commit origin/master
  Preparing metadata (setup.py) ... done


In [ ]:
import pandas as pd
import os
import re

In [ ]:
import twint

In [ ]:
c = twint.Config()
c.Search = 'Nasdaq' 
c.Limit = 50

In [ ]:
from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d')
c.Since = '2023-02-07'
c.Lang = "en"
c.Pandas = True
c.Stats = True
c.Min_likes = 20
c.Min_retweets = 1
c.Min_replies = 5

In [ ]:
twint.run.Search(c)

1623137133916479491 2023-02-08 01:50:23 +0000 <MenthorQpro> Bond market talks recession (yield curve still deeply inverted) while NASDAQ forward earning valuations decide to go after the moon - zero fear of inflation  https://t.co/nxlZkZFS7s | 7 replies 9 retweets 59 likes
1623134860733972480 2023-02-08 01:41:21 +0000 <AEHarshada> Dow recovers from Day low to close 265 pts up  Nasdaq gains 225 pts   Nikkei dn 187 pts  SGX Nifty up 63 pts  ahead of RBI's Credit policy | 7 replies 3 retweets 73 likes
1623118896839675906 2023-02-08 00:37:55 +0000 <Nigel__DSouza> Fed Chair Powell comments indicated inflation has started easing😄   Bulls love it😍  🇺🇸Markets 🚀 #DowJones +0.8% #Nasdaq +1.9% S&amp;P500 +1.3%  💲Index at 103.2 ⛽️Brent at 83.7$ 🇺🇸10 yr yield at 3.66%  😢Ebay plans to lay off 500 employees, which is about 4% of its workforce | 9 replies 3 retweets 90 likes
1623110393349672960 2023-02-08 00:04:08 +0000 <jfahmy> 8 follow through-type days for the Nasdaq Composite so far this year.  ht

In [ ]:
def column_names():
    return twint.output.panda.Tweets_df.columns
def twint_to_pd(columns):
    return twint.output.panda.Tweets_df[columns]

Index(['id', 'conversation_id', 'created_at', 'date', 'timezone', 'place',
       'tweet', 'language', 'hashtags', 'cashtags', 'user_id', 'user_id_str',
       'username', 'name', 'day', 'hour', 'link', 'urls', 'photos', 'video',
       'thumbnail', 'retweet', 'nlikes', 'nreplies', 'nretweets', 'quote_url',
       'search', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'],
      dtype='object')

In [ ]:
tweet_df = twint_to_pd(["date", "username", "tweet", "hashtags", "nlikes" , "nreplies", "nretweets"])
tweet_df.head(10)

,date,username,tweet,hashtags,nlikes,nreplies,nretweets
0,2023-02-08 01:50:23,MenthorQpro,Bond market talks recession (yield curve still...,[],59,7,9
1,2023-02-08 01:41:21,AEHarshada,Dow recovers from Day low to close 265 pts up ...,[],73,7,3
2,2023-02-08 00:37:55,Nigel__DSouza,Fed Chair Powell comments indicated inflation ...,"[dowjones, nasdaq]",90,9,3
3,2023-02-08 00:04:08,jfahmy,8 follow through-type days for the Nasdaq Comp...,[],153,8,16
4,2023-02-07 23:11:41,blackwidowbtc,1. The Dow Jones - Bullish cross ✅ 2. The #SP5...,[sp500],34,5,4
5,2023-02-07 22:17:03,JulianKlymochko,The 10-year yield has been dropping but the lo...,[],108,14,21
6,2023-02-07 22:09:57,optimumhustle,#NASDAQ interviewed @VoltInuOfficial in a bear...,"[nasdaq, volt, voltinu]",84,15,62
7,2023-02-07 22:00:31,TheMaverickWS,"2yr yield (white) vs. $QQQ (orange): As ""peak""...",[],331,52,61
8,2023-02-07 21:26:57,InverseTheCons,Nasdaq could be down 5% tomorrow and no one wi...,[],39,5,3
9,2023-02-07 21:26:41,goto_finance,🇺🇸 株高 NASDAQが2%近く上昇。パウエルFRB議長が対談で「ディスインフレのプロセス...,[],2627,7,370


In [ ]:
tweet_list = tweet_df['tweet'].tolist()

In [ ]:
import json
#Store you OpenAPI key in a json file
OPENAI_API_KEY = ''
with open('/content/OpenAI.json', 'r') as file_to_read:
    json_data = json.load(file_to_read)
    OPENAI_API_KEY = json_data["OPENAI_API_KEY"]

In [ ]:
import os
import openai

openai.api_key =  OPENAI_API_KEY

In [ ]:
i =1
tweet_str=''
for tweet in tweet_list:
   tweet_str = tweet_str + str(i) +'.'+'"'+tweet+'"'+'\n'
   i=i+1
   if i == 30:
     break

In [ ]:
prompt="Classify the sentiment in these tweets with probability:\n" + tweet_str + "\n Tweet sentiment ratings:"

In [ ]:
response = openai.Completion.create(
  model="text-davinci-003",
  prompt= prompt,
  temperature=0,
  max_tokens=120,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0
)

In [ ]:
(response["choices"][0]["text"]).split('\n')[1:]

['1. Neutral (0.5)',
 '2. Positive (0.8)',
 '3. Positive (0.7)',
 '4. Neutral (0.5)',
 '5. Positive (0.9)',
 '6. Neutral (0.5)',
 '7. Positive (0.8)',
 '8. Neutral (0.5)',
 '9. Negative (0.3)',
 '10. Positive (0.7)',
 '11. Positive (0.8)',
 '12. Positive (0.8)',
 '13. Positive (0.7)',
 '14.']

In [ ]:
tweet_df['tweet']

0     Bond market talks recession (yield curve still...
1     Dow recovers from Day low to close 265 pts up ...
2     Fed Chair Powell comments indicated inflation ...
3     8 follow through-type days for the Nasdaq Comp...
4     1. The Dow Jones - Bullish cross ✅ 2. The #SP5...
5     The 10-year yield has been dropping but the lo...
6     #NASDAQ interviewed @VoltInuOfficial in a bear...
7     2yr yield (white) vs. $QQQ (orange): As "peak"...
8     Nasdaq could be down 5% tomorrow and no one wi...
9     🇺🇸 株高 NASDAQが2%近く上昇。パウエルFRB議長が対談で「ディスインフレのプロセス...
10    Stocks ended the day near session highs, rever...
11    ⚠️BREAKING:  *DOW ENDS UP MORE THAN 250 POINTS...
12    [DB] US Close  S&amp;P500: +1.30% Nasdaq: +1.9...
13    Muy floja $AMZN  Incluso con el Nasdaq tradean...
14    10 yr yields higher today again with $TLT lowe...
15    Whomst CIO running the buy the Nasdaq on the t...
16    Would you like to learn my nasdaq chopping str...
17    ⚠️BREAKING:  *NASDAQ ERASES 2% GAIN AS U.S

In [ ]:
tweet_df['tweet'][11]

'⚠️BREAKING:  *DOW ENDS UP MORE THAN 250 POINTS, NASDAQ JUMPS 1.9% AFTER POWELL SAYS INFLATION IS DECLINING -  https://t.co/8l4s32sjs1  $DIA $SPY $QQQ 🇺🇸🇺🇸'

In [ ]:
text = ''.join(tweet_df['tweet'])

In [ ]:
response_sum = openai.Completion.create(
  model="text-davinci-003",
  prompt="{} \n\nTl;dr".format(text),
  temperature=0.7,
  max_tokens=60,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=1
)

response_sum